In [ ]:
import numpy as np
import cv2

cap0=cv2.VideoCapture(1)
cap1=cv2.VideoCapture(2)
while(True):
    #获取一帧图像
    ret0,frame1=cap0.read()
    ret1,frame2=cap1.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #转化为灰度图
    cv2.imshow('frame0',frame1)
    cv2.imshow('frame1',frame2)
    #按下q停止
    if cv2.waitKey(1)& 0xFF==ord('q'):
        break
cap0.release()
cap1.release()
cv2.destroyAllWindows()

In [5]:
import cv2
cap=cv2.VideoCapture(0)
fource=cv2.VideoWriter_fourcc('F','L','V','1')
out=cv2.VideoWriter('practice.flv',fource,20.0,(640,480))
while(True):
    ret,frame=cap.read()
    out.write(frame)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.realse()
out.realse()
cv2.destroyAllWindows()

AttributeError: 'cv2.VideoCapture' object has no attribute 'realse'

In [9]:
import numpy as np
import cv2

cap=cv2.VideoCapture('practice.flv')
fource=cv2.VideoWriter_fourcc('M','J','P','G')
fps=cap.get(cv2.CAP_PROP_FPS)
print(fps)

#视频图像长度
frame_width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#视频图像宽度
frame_height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(frame_width)
print(frame_height)

#创建VideoWriter对象
out=cv2.VideoWriter('practice_new.avi',fource,fps,(frame_width,frame_height))
while(True):
    ret,frame=cap.read()
    if ret==True:
        #水平翻转
        frame=cv2.flip(frame,1)
        out.write(frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1)& 0xFF==ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

20.0
640
480


In [17]:
import cv2

#视频文件初始化
filename='move_detect.flv'
camera=cv2.VideoCapture(filename)

#视频文件输出参数设置
out_fps=12.0#输出文件帧率
fourcc=cv2.VideoWriter_fourcc('X','V','I','D')
out1=cv2.VideoWriter('v1.MP4',fourcc,out_fps,(500,400))
out2=cv2.VideoWriter('v2.MP4',fourcc,out_fps,(500,400))

#初始化当前帧
lastframe=None
#遍历视频每一帧
while camera.isOpened():
    #读取到下一帧
    (ret,frame)=camera.read()
    #如果不能抓到一帧，说明到了视频结尾
    if not ret:
        break
    #调整该帧的大小
    frame=cv2.resize(frame,(500,400),interpolation=cv2.INTER_CUBIC)
    #如果第一帧是None，对其进行初始化
    if lastframe is None:
        lastframe=frame
        continue
    
    #计算当前帧和前帧不同
    frameDelta=cv2.absdiff(lastframe,frame)
    #当前帧设置为下一帧的前帧
    lastframe=frame.copy()
    
    #结果转为灰度图
    thresh=cv2.cvtColor(frameDelta,cv2.COLOR_BGR2GRAY)
    #图像二值化
    thresh = cv2.threshold(thresh, 25, 255, cv2.THRESH_BINARY)[1] 
    #去除图像噪声,先腐蚀再膨胀(形态学开运算) 
    thresh=cv2.erode(thresh,None,iterations=1) 
    thresh = cv2.dilate(thresh, None, iterations=2)
    #与之图像中的轮廓位置
    cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #遍历轮廓，cnt为连通区域
    for c in cnts:
        #连通区域面积小于300，忽略小轮廓，排除误差
        if cv2.contourArea(c) < 300: 
            continue 
        # 计算轮廓的边界框，在当前帧中画出该框 
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) #颜色grey，线宽2
        
    #显示当前各帧
    cv2.imshow('frame',frame)
    cv2.imshow('lastframe',lastframe)
    cv2.imshow('frameDelta',frameDelta)
    cv2.imshow('thresh',thresh)
    
    #保存视频
    out1.write(frame)
    out2.write(frameDelta) 
    
    # 如果q键被按下，跳出循环 
    if cv2.waitKey(20) & 0xFF == ord('q'): 
        break 
# 清理资源并关闭打开的窗口 
out1.release() 
out2.release() 
camera.release() 
cv2.destroyAllWindows()

In [4]:
import cv2 as cv
import numpy as np

es = cv.getStructuringElement(cv.MORPH_ELLIPSE, (10, 10))
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
cap = cv.VideoCapture("move_detect.flv")
frame1 = cap.read()[1]
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

# 视频文件输出参数设置
out_fps = 12.0  # 输出文件的帧率
fourcc = cv.VideoWriter_fourcc('M', 'P', '4', '2')
sizes = (int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT)))
out1 = cv.VideoWriter('v6.avi', fourcc, out_fps, sizes)
out2 = cv.VideoWriter('v8.avi', fourcc, out_fps, sizes)

while True:
    (ret, frame2) = cap.read()
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255)

    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    draw = cv.cvtColor(bgr, cv.COLOR_BGR2GRAY)
    draw = cv.morphologyEx(draw, cv.MORPH_OPEN, kernel)
    draw = cv.threshold(draw, 25, 255, cv.THRESH_BINARY)[1]

    contours, hierarchy = cv.findContours(draw.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if cv.contourArea(c) < 500:
            continue
        (x, y, w, h) = cv.boundingRect(c)
        cv.rectangle(frame2, (x, y), (x + w, y + h), (255, 255, 0), 2)

    cv.imshow('frame2', bgr)

    cv.imshow('draw', draw)
    cv.imshow('frame1', frame2)
    out1.write(bgr)
    out2.write(frame2)

    k = cv.waitKey(20) & 0xff
    if k == 27 or k == ord('q'):
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next

out1.release()
out2.release()
cap.release()
cv.destroyAllWindows()

In [5]:
import numpy as np
import cv2

# read the video
cap = cv2.VideoCapture('move_detect.flv')

# create the subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=500, varThreshold=100, detectShadows=False)


def getPerson(image, opt=1):

    # get the front mask
    mask = fgbg.apply(frame)

    # eliminate the noise
    line = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5), (-1, -1))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, line)
    cv2.imshow("mask", mask)

    # find the max area contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in range(len(contours)):
        area = cv2.contourArea(contours[c])
        if area < 150:
            continue
        rect = cv2.minAreaRect(contours[c])
        cv2.ellipse(image, rect, (0, 255, 0), 2, 8)
        cv2.circle(image, (np.int32(rect[0][0]), np.int32(rect[0][1])), 2, (255, 0, 0), 2, 8, 0)
    return image, mask


while True:
    ret, frame = cap.read()
    cv2.imwrite("input.png", frame)
    cv2.imshow('input', frame)
    result, m_ = getPerson(frame)
    cv2.imshow('result', result)
    k = cv.waitKey(20) & 0xff
    if k == 27 or k == ord('q'):
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next

cap.release()
cv2.destroyAllWindows()